### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

In [13]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")


In [3]:
!pip install langchain_groq


   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 2/2 [langchain_groq]



In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025D76E069B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025D76E3CBE0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family.)\n* **Salut, ça va ?** (This is a very casual way to say it.)\n\n\nLet me know if you\'d like to know more French greetings!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 21, 'total_tokens': 128, 'completion_time': 0.194545455, 'prompt_time': 0.001328769, 'queue_time': 0.24844162, 'total_time': 0.195874224}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f0f4e2f3-da7d-42fc-8889-5068bc3bef50-0', usage_metadata={'input_tokens': 21, 'output_tokens': 107, 'total_tokens': 128})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family.)\n* **Salut, ça va ?** (This is a very casual way to say it.)\n\n\nLet me know if you\'d like to know more French greetings!\n'

In [9]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, singular)\n* **Salut, comment allez-vous ?** (Informal, plural) \n\nThe best option depends on the context and your relationship with the person you\'re speaking to. \n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [11]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [ ]:
result.to_messages()

In [12]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'